In [ ]:
pip install -U scikit-learn

In [ ]:
pip install pandas

In [ ]:
"""
Create a model relating all of the features in the https://www.kaggle.com/datasets/brsdincer/star-type-classification set
"""

"""
Make a "summary" of the features in the dataset
1. Accuracy of model(s)
    a. Temperature + Luminosity, T + L + R, T + L + R + M, L + R + M -> a. color (Red, Blue White, White, Yellowish, Pale yellow orange, Blue, Whitish, Yellow-White, Orange, Orange-Red,)
    b. Temperature + Luminosity -> Spectral Class
    c. Temperature + Luminosity -> Type
2. Temperature and Luminosity ranges for each star type
"""

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
#import matplotlib.pyplot as plt
#import seaborn as sns

In [5]:
og_df = pd.DataFrame(pd.read_csv('Stars.csv'))

#Change colors related to yellow and white to Yellowish White
og_df = og_df.replace(to_replace="White Yellow", value="yellow-white")
og_df = og_df.replace(to_replace="Yellowish", value="yellow-white")
og_df = og_df.replace(to_replace="White-Yellow", value="yellow-white")
og_df = og_df.replace(to_replace="White Yellow", value="yellow-white")
og_df = og_df.replace(to_replace="Whitish", value="yellow-white")
og_df = og_df.replace(to_replace="White", value="yellow-white")
og_df = og_df.replace(to_replace="white Yellow", value="yellow-white")
og_df = og_df.replace(to_replace="yellowish", value="yellow-white")
og_df = og_df.replace(to_replace="white", value="yellow-white")
og_df = og_df.replace(to_replace="yellow-white", value="Yellowish White")
#Change colors related to orange to Orange
og_df = og_df.replace(to_replace="Pale yellow orange", value="Orange")
og_df = og_df.replace(to_replace="Orange-Red", value="Orange")
#Change Blue-white to Blue White
og_df = og_df.replace(to_replace="Blue-white", value="Blue White")
og_df = og_df.replace(to_replace="Blue-White", value="Blue White")

#1a
#Create new datasets with the Temperature, Luminosity, and Color features
T = og_df[['Temperature']].copy()
L = og_df[['L']].copy()
color = og_df[['Color']].copy()
color = color.to_numpy()
radius = og_df[['R']].copy()
absolute_magnitude = og_df[['A_M']].copy()

#1st problem - Which model, because none were working
#Solution - turn the color df into a numpy array and use .ravel()
temp_train, temp_test, l_train, l_test, color_train, color_test, radi_train, radi_test, mag_train, mag_test = train_test_split(T, L, color, radius, absolute_magnitude, test_size = 0.2)
TL_train = pd.concat([temp_train, l_train], axis=1)
TL_test = pd.concat([temp_test, l_test], axis=1)
color_train = color_train.ravel()

print("TESTING MODELS")
print('------------------------------------------------------------------------------------------------------------------')
#Testing different models, K Nearest Neighbors
n=8
TLC = KNeighborsClassifier(n_neighbors = n)
TLC.fit(TL_train, color_train)
TLC_pre = TLC.predict(TL_test)

print("Accuracy Score of KNN (Temperature+Luminosity->Color) is {}".format(accuracy_score(color_test, TLC_pre)))
#Problem 2 - Original KNN Score is always around 0.6-0.7 which can be improved
#Try 1: Testing with only temperature, as I think the problem may be in the Luminosity feature
#Try 2: Testing Luminosity, inconclusive
#Try 3: realized that the replace statements above weren't working
#Solution: had to equate the replace statements (df.replace to df = df.replace)
#Brought KNN Score to 0.65-0.8
TC = KNeighborsClassifier(n_neighbors = n)
TC.fit(temp_train, color_train)
TC_pre = TC.predict(temp_test)

print("Accuracy Score of KNN (Temperature->Color) is {}".format(accuracy_score(color_test, TC_pre)))

LC = KNeighborsClassifier(n_neighbors = n)
LC.fit(l_train, color_train)
LC_pre = LC.predict(l_test)

print("Accuracy Score of KNN (Luminosity->Color) is {}".format(accuracy_score(color_test,LC_pre)))

print('------------------------------------------------------------------------------------------------------------------')

#Next problem: Still wasn't satisfied with the accuracy of the model, trying out Random Forest Model
rTLC = RandomForestClassifier(max_depth = 4)
rTLC.fit(TL_train, color_train)
rTLC_pre = rTLC.predict(TL_test)

print("Accuracy Score of Random Forest (Temperature+Luminosity->Color) is {}".format(accuracy_score(color_test, rTLC_pre)))

rTC = RandomForestClassifier(max_depth = 4)
rTC.fit(temp_train, color_train)
rTC_pre = rTC.predict(temp_test)

print("Accuracy Score of Random Forest (Temperature->Color) is {}".format(accuracy_score(color_test, rTC_pre)))

rLC = RandomForestClassifier(max_depth = 4)
rLC.fit(l_train, color_train)
rLC_pre = rLC.predict(l_test)

print("Accuracy Score of Random Forest (Luminosity->Color) is {}".format(accuracy_score(color_test,rLC_pre)))

#Temp and L model accuracy ranges from 0.8 - 0.95, which is good
#The other two, just temp and just L, are below that range
#More features = better
print('------------------------------------------------------------------------------------------------------------------')
# I want to see if the model can be even more accurate
# Trying Decision Tree
dTLC = DecisionTreeClassifier(max_depth = 4, criterion = 'entropy')
dTLC.fit(TL_train, color_train)
dTLC_pre = dTLC.predict(TL_test)

print("Accuracy Score of Decision Tree (Temperature+Luminosity->Color) is {}".format(accuracy_score(color_test, dTLC_pre)))

dTC = RandomForestClassifier(max_depth = 4)
dTC.fit(temp_train, color_train)
dTC_pre = dTC.predict(temp_test)

print("Accuracy Score of Decision Tree (Temperature->Color) is {}".format(accuracy_score(color_test, dTC_pre)))

dLC = RandomForestClassifier(max_depth = 4)
dLC.fit(l_train, color_train)
dLC_pre = dLC.predict(l_test)

print("Accuracy Score of Decision Tree (Luminosity->Color) is {}".format(accuracy_score(color_test,dLC_pre)))

#Decision Tree accuracy is between KNN and Random Forest (0.75 - 0.85)
#Random Forest is the algorithm I should use
print('------------------------------------------------------------------------------------------------------------------')

#train sets for temp, luminosity, and color already made
#make for Radius and absolute magnitude

TLR_train = pd.concat([temp_train, l_train, radi_train], axis=1)
TLR_test = pd.concat([temp_test, l_test, radi_test], axis=1)

rTLRC = RandomForestClassifier(max_depth = 4)
rTLRC.fit(TLR_train, color_train)
rTLRC_pre = rTLRC.predict(TLR_test)

print("Accuracy Score of Random Forest (Temperature+Luminosity+Radius->Color) is {}".format(accuracy_score(color_test, rTLRC_pre)))
# pd.set_option('display.max_rows', None)
# display(TLR_test)

TESTING MODELS
------------------------------------------------------------------------------------------------------------------
Accuracy Score of KNN (Temperature+Luminosity->Color) is 0.7916666666666666
Accuracy Score of KNN (Temperature->Color) is 0.6875
Accuracy Score of KNN (Luminosity->Color) is 0.6666666666666666
------------------------------------------------------------------------------------------------------------------
Accuracy Score of Random Forest (Temperature+Luminosity->Color) is 0.875
Accuracy Score of Random Forest (Temperature->Color) is 0.7083333333333334
Accuracy Score of Random Forest (Luminosity->Color) is 0.6666666666666666
------------------------------------------------------------------------------------------------------------------
Accuracy Score of Decision Tree (Temperature+Luminosity->Color) is 0.75
Accuracy Score of Decision Tree (Temperature->Color) is 0.7291666666666666
Accuracy Score of Decision Tree (Luminosity->Color) is 0.6666666666666666
----

Extra Sources I Used:
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html                                   (and other pages on the Pandas Documentation)
https://www.kaggle.com/datasets/brsdincer/star-type-classification                                           (Dataset)
stackoverflow.com                                                                                            (Reference)
https://medium.com/codex/credit-card-fraud-detection-with-machine-learning-in-python-ac7281991d87            (Reference)
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html              (and other pages on the Sklearn Documentation)